# Deterministic Global Optimization for Concentrating Solar Thermal Hybridization

This example comes from M.D. Stuber. A Differentiable Model for Optimizing Hybridization of Industrial Process Heat Systems with Concentrating Solar Thermal Power. _Processes_, 6(7), 76 (2018) DOI: [10.3390/pr6070076](https://doi.org/10.3390/pr6070076)

In this example, we seek to determine the optimal thermal energy storage capacity and parabolic trough solar array aperture area that maximizes the lifecycle savings associated with augmenting a conventional natural gas industrial process heat system.  Here, we use user-defined functions, the JuMP modeling language, the EAGO spatial branch-and-bound algorithm with custom upper- and lower-bounding procedures, and the IPOPT algorithm for solving the bounding subproblems.

We will solve the optimal design problem for Firebaugh, CA with the commercial fuel rate and constant industrial process heat demand ($\xi=0$).

Note: This example corresponds to a $\bar{q}_p=10^4$ kW thermal demand.  In the paper, it is stated that a $\bar{q}_p=10^5$ kW thermal demand was studied.  However, this is an error as all studies in the paper were conducted for a $\bar{q}_p=10^4$ kW thermal demand.

In [1]:
using JuMP, EAGO, Ipopt;

We will also use the CSV package because our solar resource data (downloaded from the NSRDB) is in a CSV file.

In [2]:
using CSV;

┌ Info: Recompiling stale cache file C:\Users\wilhe\.julia\compiled\v1.2\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1240


The code is organized as follows:  
1. We import the solar resource data for the region we are concerned with.  By default, this is the typical meteorological year (TMY) hourly data (8760 data points) which includes the geographic information (coordinates and timezone) as well as the direct normal irradiance (DNI).  Together, we can model the process of concentrating the solar radiation and collecting it as heat in our system.  
2. Once we have the resource data, we call "PTCmodel.jl" to simulate the specific thermal power of the solar concentrating technology (a parabolic trough in this case) in units of kW/m$^2$ given our geographic region and incident angle data.  This function calls "solarAngles.jl" to calculate the angles of the direct solar radiation incident to the concentrator aperture with respect to each hour in the TMY data.
4. We define a function closure for the solar fraction calculated by "iphProcessSmooth.jl", which simulates the performance of the full solar concentrator and thermal storage system hybridized with the industrial process heat system.  The solar fraction is calculated by Eq. 12 in the paper.
5. We define the custom upper- and lower-bounding optimization subproblems for the spatial branch-and-bound algorithm.  The lower-bounding problem uses the convex capital model for its objective function (Eq. 16 in the paper), and the upper-bounding problem uses the nonconvex capital model as its objective function (Eq. 14 in the paper).
6. We set up the JuMP model with the EAGO optimizer (with custom bounding procedures) and we solve it.

In [3]:
include("solarAngles.jl")
include("PTCmodel.jl")
include("smoothMinMaxAbs.jl")
include("iphProcessSmooth.jl")
include("lifecycleCost.jl")
# Step 1: read the data into a table and extract the appropriate data into a vector
input_file = "FirebaughTMY_Julia.csv"
solData = CSV.read(input_file)
yData = convert(Array{Float64,1},solData[:,7])

# Step 2: get the specific thermal power potential for the region [kW/m^2]
q = PTCmodel(yData,-8,36.85,-120.46)

# Step 3: define the solar fraction closure
SolarFrac(xts,xa) = iphProcessSmooth(q,xts,xa);

Step 4 is a bit more complicated as we need to define the custom bounding procedures for the branch-and-bound algorithm.  We first define an extension type, this extension type will allow EAGO to dispatch to the custom routines we'll subsequently define.

In [4]:
struct SolarExt <: EAGO.ExtensionType end

First, we define the lower-bounding problem with the convex capital model.

In [25]:
# Lower Problem Definition
import EAGO.lower_problem!
function lower_problem!(t::SolarExt, opt::EAGO.Optimizer)

    # Get active node
    n = opt._current_node
    
    # Creates model, adds variables, and register nonlinear expressions
    m = JuMP.Model(JuMP.with_optimizer(Ipopt.Optimizer,tol=1.0e-3))
    xL = n.lower_variable_bounds; xU = n.upper_variable_bounds
    @variable(m, xL[i] <= x[i=1:2] <= xU[i])
    
    # define convex lifecycle savings objective function cover
    flcsC(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :convex)
    JuMP.register(m, :flcsC, 2, flcsC, autodiff=true)
    JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define nonlinear function
    @NLobjective(m, Min, -flcsC(x[1], x[2]))
    @NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= 0.0)# declare constraints
    JuMP.optimize!(m)

    # Get primal status, termination status, determine if a global solution was obtained
    termination_status = JuMP.termination_status(m)
    primal_status = JuMP.primal_status(m)
    feasible_flag = EAGO.is_feasible_solution(termination_status, primal_status)

    # Interpret status codes for branch-and-bound
    if feasible_flag
        opt._lower_objective_value = 1.0*(JuMP.objective_value(m)-1e-4)
        opt._lower_solution = JuMP.value.(x)
        opt._lower_feasibility = true
        opt._cut_add_flag = false
    else
        opt._lower_feasibility = false
        opt._lower_objective_value = -Inf
        opt._cut_add_flag = false
    end
    return
end

lower_problem! (generic function with 3 methods)

Now, we define the upper-bounding problem with the nonconvex capital pricing model.

In [26]:
# Upper Problem Definition
import EAGO.upper_problem!
function EAGO.upper_problem!(t::SolarExt, opt::EAGO.Optimizer)
    
    # Get active node
    n = opt._current_node

    # Creates model, adds variables, and register nonlinear expressions
    m = JuMP.Model(JuMP.with_optimizer(Ipopt.Optimizer,tol=1.0e-3))
    xL = n.lower_variable_bounds; xU = n.upper_variable_bounds
    @variable(m, xL[i] <= x[i=1:2] <= xU[i])
    
    # define lifecycle savings objective function cover
    flcs(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :nonconvex)
    JuMP.register(m, :flcs, 2, flcs, autodiff=true)
    JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define nonlinear function
    @NLobjective(m, Min, -flcs(x[1], x[2]))
    @NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= 0.0)# declare constraints
    JuMP.optimize!(m)

    # Get primal status, termination status, determine if a global solution was obtained
    termination_status = JuMP.termination_status(m)
    primal_status = JuMP.primal_status(m)
    feasible_flag = EAGO.is_feasible_solution(termination_status, primal_status)

    # Interpret status codes for branch-and-bound
    if feasible_flag
        opt._upper_objective_value = 1.0*JuMP.objective_value(m)
        opt._upper_solution = JuMP.value.(x)
        opt._upper_feasibility = true
    else
        opt._upper_feasibility = false
        opt._upper_objective_value = -Inf
        opt._cut_add_flag = false
    end
    return 
end

Disable unecessary routines.

In [27]:
import EAGO: preprocess!, postprocess!, cut_condition
function EAGO.preprocess!(t::SolarExt, x::Optimizer)
    x._preprocess_feasibility = true
    return
end
function EAGO.postprocess!(t::SolarExt, x::Optimizer)
    x._postprocess_feasibility = true
    return
end
EAGO.cut_condition(t::SolarExt, x::Optimizer) = false

Now, we define the JuMP model and the variables (and bounds).

In [28]:
m = JuMP.Model(with_optimizer(EAGO.Optimizer, relative_tolerance=1e-2, verbosity=1, obbt_depth = 0, dbbt_depth = 0, cp_depth = 0,
                              branch_variable = Bool[true; true; true; true], ext_type = SolarExt()))

x_L = [0.001,000.01]
x_U = [16.0,60000.0]
@variable(m, x_L[i] <= x[i=1:2] <= x_U[i]);

Now, let's solve the problem and print the results.

In [29]:
JuMP.optimize!(m)

println("xts* = ", JuMP.value(x[1]), " xa* = ",
         JuMP.value(x[2])," f* = ",-1.0*JuMP.objective_value(m)," SF* = ",
         SolarFrac(JuMP.value(x[1]),JuMP.value(x[2])))
TermStatus = JuMP.termination_status(m)
PrimStatus = JuMP.primal_status(m)
println("Algorithm terminated with a status of $TermStatus and a result code of $PrimStatus")

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

First Solution Found at Node 7
UBD = -7.317666312971923e6
Solution is :
    X[1] = 11.722361250304994
    X[2] = 43615.190600813985
xts* = 11.722361250304994 xa* = 43615.190600813985 f* = 7.317666312971923e6 SF* = 0.6980196901665338
Algorithm terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT


So, we find the guaranteed global optimal solution is $x_{ts}^*=11.72$ h and $x_{a}^*=43,615.2$ m$^2$ with a solar fraction of $SF_s^*=0.698$ and an optimal solution value of $f^*_{disc}=7.32$ million dollars.  This is exactly what is found in Table 1 in the paper for Firebaugh, CA and the concave capital cost model.